In [4]:
# project 2 solutions

# theoretical 7-15
5 % 0

ZeroDivisionError: integer division or modulo by zero

In [ ]:
import random

DEF_DIFFIC = 10
MAX_TRIALS = 2000
MAX_VALUE = 99
OPERATORS = "+-*%"
EQUALITY = "="
DIGITS = "0123456789"
NOT_POSSIBLE = "No FoCdle found of that difficulty"


def random_number():
    """
    Generates a random number from [1, 99] with no leading zeros
    """

    return str(random.randint(1, 99))

def random_operator():
    """
    Generates a random operator
    """

    return random.choice(OPERATORS)

def random_expression():
    """
    Generates a random valid LHS expression
    """

    return random_number() + random_operator() \
            + random_number() + random_operator() + random_number()



def create_secret(difficulty=DEF_DIFFIC):
    '''
    Use a random number function to create a FoCdle instance of length 
    `difficulty`. The generated equation will be built around three values 
    each from 1 to 99, two operators, and an equality.
    '''

    for _ in range(MAX_TRIALS):
        # create valid expression
        expression = random_expression()

        # compute outcome and discard if outcome is invalid
        outcome = eval(expression)
        if outcome <= 0:
            continue

        # compute secret and discard if length incorrect
        secret = f'{expression}={outcome}'
        if len(secret) != difficulty:
            continue

        # we have found one!
        return secret

    # we failed to find one :(
    return NOT_POSSIBLE




In [1]:
GREEN = "green"
YELLO = "yellow"
GREYY = "grey"

def index_equation(equation: str):
    """
    Index the equation by returning the frequencies of all its characters
    """

    frequency = {}
    for c in equation:
        frequency[c] = frequency.get(c, 0) + 1

    return frequency


def set_colors(secret: str, guess: str):
    '''
    Compares the latest `guess` equation against the unknown `secret` one. 
    Returns a list of three-item tuples, one tuple for each character position 
    in the two equations:
        -- a position number within the `guess`, counting from zero;
        -- the character at that position of `guess`;
        -- one of "green", "yellow", or "grey", to indicate the status of
           the `guess` at that position, relative to `secret`.
    The return list is sorted by position.
    '''

    # index equation
    secret_freq = index_equation(secret)

    # mark greens
    for secret_char, guess_char in zip(secret, guess):
        if guess_char == secret_char:
            # the character is correctly guessed, so remove it from freq
            secret_freq[guess_char] -= 1

    # fill output
    colors = []
    for i in range(len(guess)):
        secret_char = secret[i]
        guess_char = guess[i]

        if guess_char == secret_char:
            # the character is correctly guessed, so return GREEN
            colors.append((i, guess_char, GREEN))

        elif guess_char in secret_freq and secret_freq[guess_char] > 0:
            # the character is incorrectly placed,
            # so remove it from freq and return YELLOW
            secret_freq[guess_char] -= 1
            colors.append((i, guess_char, YELLO))

        else:
            # the character does not exist in secret, so return GREY
            colors.append((i, guess_char, GREYY))

         
    return colors



# set_colors("1+2*3=7", "1***3*+")
set_colors("543567", "355554")

[(0, '3', 'yellow'),
 (1, '5', 'yellow'),
 (2, '5', 'grey'),
 (3, '5', 'green'),
 (4, '5', 'grey'),
 (5, '4', 'yellow')]

In [192]:
from typing import List, Tuple

GREEN = "green"
YELLO = "yellow"
GREYY = "grey"

OPERATORS = "+-*%"
EQUALITY = "="
DIGITS = "0123456789"

CHARS = [*DIGITS, *OPERATORS, *EQUALITY]

def build_index(info: List[List[Tuple[int, str, str]]]):
    """
    Build a character lookup table from color infos
    """

    lookup = {}
    
    # initialize the table
    for key in CHARS:
        lookup[key] = {
            # correct indices
            "correct": set(),
            # incorrect indices
            "incorrect": set(),
            # minimum and maximum frequencies
            "minimum": 0,
            "maxed": False
        }
    
    # for each past guess
    for guess in info:

        # this stores the possible character freqs (ignoring greys)
        character_freqs = {}
        maxed_characters = set()

        for index, key, color in guess:
            if color == GREEN:
                # add index to correct
                lookup[key]["correct"].add(index)
                character_freqs[key] = character_freqs.get(key, 0) + 1
            elif color == YELLO:
                # add index to incorrect
                lookup[key]["incorrect"].add(index)
                character_freqs[key] = character_freqs.get(key, 0) + 1
            else:
                # add index to incorrect, and indicate maximum reached
                lookup[key]["incorrect"].add(index)
                maxed_characters.add(key)
        
        # update lookup minmax values
        for key in CHARS:
            # means maximum is reached, min is correct
            if lookup[key]["maxed"]:
                continue

            # set minimum to be the highest estimated minimum
            lookup[key]["minimum"] = max(lookup[key]["minimum"], character_freqs.get(key, 0))

            # indicate that maximum characters are reached
            if key in maxed_characters:
                lookup[key]["maxed"] = True
                

    return lookup

def passes_restrictions(guess: str, all_info: List[List[Tuple[int, str, str]]]):
    '''
    Tests a `guess` equation against `all_info`, a list of known restrictions, 
    one entry in that list from each previous call to set_colors(). Returns 
    True if that `guess` complies with the collective evidence imposed by 
    `all_info`; returns False if any violation is detected. Does not check the 
    mathematical accuracy of the proposed candidate equation.
    '''

    index = build_index(all_info)

    # create guess character frequencies
    guess_freqs = {}
    for c in guess:
        guess_freqs[c] = guess_freqs.get(c, 0) + 1


    # the most unique characters the guess could have
    uniques_upperbound = len(guess)

    # check that all character infos are satisfied
    for key in CHARS:
        lookup = index[key]

        # check that all green locations are filled
        for correct in lookup["correct"]:
            if guess[correct] != key:
                return False

        # check that all yellow/grey locations are cleared
        for incorrect in lookup["incorrect"]:
            if guess[incorrect] == key:
                return False
        
        if lookup["maxed"]:  # if maxed, check frequency equality
            if guess_freqs.get(key, 0) != lookup["minimum"]:
                return False
        else:  # if not maxed, check freqency using lower bound
            if guess_freqs.get(key, 0) < lookup["minimum"]:
                return False
            
        # trim duplicate characters
        # zero/one minimum indicates a potential unique
        # more than one minimum indicates minimum-1 duplicates
        uniques_upperbound -= max(0, (lookup["minimum"]) - 1)

    # check that guess uniques are under the upperbound
    uniques = len(set(guess))
    if uniques > uniques_upperbound:
        return False

    return True


    
    

In [83]:
info = [[(0, '1', 'yellow'), (1, '3', 'yellow'), (2, '+', 'green'), (3, '1', 'grey'), 
(4, '2', 'yellow'), (5, '-', 'grey'), (6, '8', 'grey'), (7, '=', 'green'), 
(8, '1', 'grey'), (9, '7', 'yellow')], [(0, '7', 'grey'), (1, '2', 'yellow'), 
(2, '+', 'green'), (3, '3', 'grey'), (4, '1', 'yellow'), (5, '%', 'grey'), 
(6, '6', 'grey'), (7, '=', 'green'), (8, '7', 'green'), (9, '3', 'green')], 
[(0, '2', 'green'), (1, '1', 'yellow'), (2, '+', 'green'), (3, '2', 'yellow'), 
(4, '0', 'grey'), (5, '*', 'yellow'), (6, '4', 'yellow'), (7, '=', 'green'), 
(8, '5', 'yellow'), (9, '5', 'grey')]]

# info = [[(0, '9', )]]
# passes_restrictions("25+4*12=73", info)
create_guess(info, 10)

'28+%521=73'

In [160]:
import random
from typing import List, Tuple

DEF_DIFFIC = 10

GREEN = "green"
YELLO = "yellow"
GREYY = "grey"

OPERATORS = "+-*%"
EQUALITY = "="
DIGITS = "0123456789"

CHARS = [*DIGITS, *OPERATORS, *EQUALITY]


def build_index(info: List[List[Tuple[int, str, str]]]):
    """
    Build a character lookup table from color infos
    """

    lookup = {}
    
    # initialize the table
    for key in CHARS:
        lookup[key] = {
            # correct indices
            "correct": set(),
            # incorrect indices
            "incorrect": set(),
            # minimum and maximum frequencies
            "minimum": 0,
            "maxed": False
        }
    
    # for each past guess
    for guess in info:

        # this stores the possible character freqs (ignoring greys)
        character_freqs = {}
        maxed_characters = set()

        for index, key, color in guess:
            if color == GREEN:
                # add index to correct
                lookup[key]["correct"].add(index)
                character_freqs[key] = character_freqs.get(key, 0) + 1
            elif color == YELLO:
                # add index to incorrect
                lookup[key]["incorrect"].add(index)
                character_freqs[key] = character_freqs.get(key, 0) + 1
            else:
                # add index to incorrect, and indicate maximum reached
                lookup[key]["incorrect"].add(index)
                maxed_characters.add(key)
        
        # update lookup minmax values
        for key in CHARS:
            # means maximum is reached, min is correct
            if lookup[key]["maxed"]:
                continue

            # set minimum to be the highest estimated minimum
            lookup[key]["minimum"] = max(
                lookup[key]["minimum"], 
                character_freqs.get(key, 0)
            )

            # indicate that maximum characters are reached
            # this is last as we want minimum to be updated
            if key in maxed_characters:
                lookup[key]["maxed"] = True
                
    return lookup


def create_guess(all_info, difficulty=DEF_DIFFIC):
    '''
    Takes information built up from past guesses that is stored in `all_info`, 
    and uses it as guidance to generate a new guess of length `difficulty`.
    '''
    index = build_index(all_info)

    # remove all grey characters & create valid position lookups
    valid = []
    valid_positions = {}
    for key, lookup in index.items():
        # ignore the grey character
        if lookup["maxed"] and lookup["minimum"] == 0:
            continue
        
        valid.append((key, lookup))

        # add to valid position lookups
        for index in lookup["correct"]:
            valid_positions[index] = key

    # output variables (this is the guess)
    output = ""
    output_frequencies = {}

    # generate character for each index
    for i in range(difficulty):
        
        # use valid positions to immediately fill position
        if i in valid_positions:
            key = valid_positions[i]
            output += key
            output_frequencies[key] = output_frequencies.get(key, 0) + 1
            continue

        # iterate through all characters to find valid characters
        choices = []
        for key, lookup in valid:
            # filter yellows
            if i in lookup['incorrect']:
                continue

            # don't add if we exceeded the maximum length of this character
            too_much = lookup['maxed'] and \
                lookup['minimum'] <= output_frequencies.get(key, 0)
            if not too_much:
                choices.append(key)
        
        # failsafe if no choices are found 
        if len(choices) == 0:
            choices = CHARS

        # pick a random character and append
        key = random.choice(choices)
        output += key
        output_frequencies[key] = output_frequencies.get(key, 0) + 1
    
    
    return output


In [309]:
import random
from typing import Set

DEF_DIFFIC = 10

NUMS = "0123456789"
OPERATORS = "+-*%"
PURE_CHARS = "0123456789+-*%"

def generate_initial_guess(difficulty):
    nums = list(NUMS)
    random.shuffle(nums)

    ops = list(OPERATORS)
    random.shuffle(ops)

    random.shuffle(nums)
    sliced = nums[:difficulty]


    if difficulty == 7:
        sliced[-2] = '='
        sliced[1] = ops[0]
        sliced[3] = ops[1]
    elif difficulty == 8:
        sliced[-2] = '='
        sliced[1] = ops[0]
        sliced[2] = ops[1]
        sliced[3] = ops[2]
        sliced[4] = ops[3]
    elif difficulty == 15:
        sliced[-3] = '='
        sliced[2] = ops[0]
        sliced[5] = ops[1]
    else:
        sliced[-2] = '='
        sliced[-3] = '='
        sliced[1] = ops[0]
        sliced[2] = ops[1]
        sliced[4] = ops[2]
        sliced[5] = ops[3]

    return ''.join(sliced)


def generate(partial: List[str], ops, index):
    blacklisted = set()
    whitelisted = set(NUMS) - blacklisted
    exist = {}

    op_whitelist = []
    for key, lookup in index.items():
        if key not in NUMS:
            if not lookup['maxed'] and lookup['minimum'] == 0:
                op_whitelist.append(key)
            continue

        if lookup['maxed']:
            if lookup['minimum'] == 0:
                blacklisted.add(key)
            else:
                exist[key] = lookup['minimum']
        

    size = len(partial) - 2

    gap1 = min(ops)
    gap2 = max(ops) - min(ops) - 1
    ints = [gap1, gap2, size - gap1 - gap2]
        
    output = [*partial]
    # fill digits
    for i in range(3):
        # random element
        digit = random.choice(list(whitelisted - {0}))

        if digit in exist:
            exist[digit] -= 1

            if exist[digit] == 0:
                whitelist -= {digit}

        output[ints[i]] = digit
        

        if ints[i] == 2:
            digit2 = random.choice(list(whitelisted))

            if digit2 in exist:
                exist[digit2] -= 1

                if exist[digit2] == 0:
                    whitelist -= {digit2}

            output[ints[i]+1] = digit2

    # fill operators
    
    output[min(ops)] = random.choice(op_whitelist)
    output[max(ops)] = random.choice(op_whitelist)

    return "".join(output)

def derive_operators(index, difficulty) -> Set[int]:
    if difficulty == 7:
        return {1, 3}
    if difficulty == 15:
        return {2, 5}

    must = set()
    for key in index:
        if key not in OPERATORS:
            continue
        
        must |= index[key]['correct']

    if len(must) == 2:
        return must
    
    return None


def create_better_guess(all_info, difficulty=DEF_DIFFIC):
    '''
    Takes information built up from past guesses that is stored in `all_info`, 
    and uses it as guidance to generate a new guess of length `difficulty`. 
    '''

    # NOTE THAT PASS_RESTRICTIONS IS NOT CALLED


    # theoretical 7-15
    if len(all_info) == 0:
        return generate_initial_guess(difficulty)
    

    index = build_index(all_info)

    # find correct operators
    # estimate positions
    operators = derive_operators(index, difficulty)

    if operators is None:
        return create_guess(all_info, difficulty)


    # format
    # num|op|num|op|num = |num|

    # equality position
    result_length = difficulty - list(index['=']['correct'])[0] - 1
    expression_length = difficulty - result_length - 1

    output = [None] * difficulty

    # fill absolute correct ones
    for key, lookup in index.items():
        for i in lookup["correct"]:
            output[i] = key
            lookup["minimum"] -= 1

    # fill the rest
    while True:
        guess = generate(output[:expression_length], operators, index)
        
        result = eval(guess)
        if result <= 0 or len(str(result)) != result_length:
            continue

        return guess + "=" + str(result)

        


        



In [310]:
def test(secret):
    guesses = 0
    info = []
    while True:
        guess = create_better_guess(info, len(secret))
        if secret == guess:
            break
        
        print(f"{guess=}")
        print(f"{secret=}")
        colors = set_colors(secret, guess)
        info.append(colors)
        guesses += 1


    return guesses

secret = create_secret(8)
total = 0
n = 1
for _ in range(n):
    total += test(secret)

print(total / n)

guess='8*-%+4=0'
secret='2%89*3=6'
guess='382*85=6'
secret='2%89*3=6'
guess='*9=879=6'
secret='2%89*3=6'
guess='6336*6=6'
secret='2%89*3=6'
guess='%68=*%=6'
secret='2%89*3=6'
guess='9%83**=6'
secret='2%89*3=6'


IndexError: list index out of range

guess='4*%-+7=5'
secret='85*6%7=6'


KeyError: '6'

In [ ]:
info = set_colors("05028324-9", "25+4*12=73")
build_index([info])

In [256]:
size = 3

small = size // 3
large = size // 3 + size % 3
medium = size - small - large

(small, medium, large)

(1, 1, 1)